In [69]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import joblib
import numpy as np

In [156]:
# load data
claims = pd.read_csv('../CoverMyMeds_data/dim_claims.csv')

In [233]:
# Dash layout

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

markdown_text = '''
#### MEDWISE
An app to help you get the meds you need'''

app.layout = html.Div([
    dcc.Markdown(children = markdown_text, style={'textAlign': 'center','fontSize': '16px'}),
    html.Label('Hello! Please select one of the options below:',style={'textAlign': 'center'}),
    dcc.RadioItems(id='Claim_PA', options=
                  [{'label': 'Check if the claim will get approved','value':'Claim'},
                  {'label' : 'Check if the PA will get approved', 'value': 'PA'}],style={'textAlign': 'center'}),
    html.Br(),
    html.Div(id ='Claim_PA_result',children=[],style={'align-items': 'center'})
 
])

@app.callback(
    Output(component_id='Claim_PA_result', component_property='children'),
    Input(component_id='Claim_PA', component_property='value'),
    State('Claim_PA_result', 'children'))
    
def Update_Claim_PA (Claim_PA_value,children):
    if Claim_PA_value == 'Claim':
        new_element_claim = html.Div([
            html.Br(),
            html.Label("Select Drug Type"),
            dcc.Dropdown(id="Drug_Claim" , options=[
                  {'label': 'Drug A', 'value': 'A'},
                  {'label': 'Drug B', 'value': 'B'},
                  {'label': 'Drug C', 'value': 'C'}
              ]),
            html.Br(),
            
            html.Label("Select Payer"),
            dcc.Dropdown(id="Payer_Claim" , options=[
                  {'label': '417380', 'value': '417380'},
                  {'label': '999001', 'value': '999001'},
                  {'label': '417740', 'value': '417740'},
                  {'label': '417614', 'value': '417614'}
              ]),
            html.Br(),
            
            html.Div(id='result_Claim')
        ])
        
        return new_element_claim

    if Claim_PA_value == 'PA' :
        new_element_PA = html.Div([
            html.Br(),
            html.Label("Select Drug Type"),
            dcc.Dropdown(id="Drug_PA" , options=[
                  {'label': 'Drug A', 'value': 'A'},
                  {'label': 'Drug B', 'value': 'B'},
                  {'label': 'Drug C', 'value': 'C'}
              ]),
            html.Br(),
            
            html.Label("Select Payer"),
            dcc.Dropdown(id="Payer_PA" , options=[
                  {'label': '417380', 'value': 'C1'},
                  {'label': '999001', 'value': 'C2'},
                  {'label': '417740', 'value': 'C3'},
                  {'label': '417614', 'value': 'C4'}
              ]),
            html.Br(),
            
#             html.Label("Select rejection code"),
#             dcc.Dropdown(id="reject_code", options=[
#                 {'label': '70', 'value': '70'},
#                 {'label': '75', 'value': '75'},
#                 {'label':'76', 'value': '76'}
#             ]),
#             html.Br(),
            
            html.Label("Select Patient Information"),
            html.Label("Is the diagnosis correct?"),
            dcc.RadioItems(id="Correct_Diagnosis", options=[
                {'label':'Yes', 'value':'1'},
                {'label':'No','value':'0'}],labelStyle={'display': 'inline-block'}),
            html.Br(),
            
            html.Label("Did the patient try and fail alternate medication?"),
            dcc.RadioItems(id="Tried_Failed",options=[
                {'label':'Yes', 'value':'1'},
                {'label':'No', 'value':'0'}
            ],labelStyle={'display': 'inline-block'}),
            html.Br(),
            
            html.Label("Did the patient show contraindication to this medicine?"),
            dcc.RadioItems(id="Contraindication", options=[
                {'label':'Yes', 'value':'1'},
                {'label':'No','value':'0'}
            ],labelStyle={'display': 'inline-block'}),
            
            
            html.Br(),
            html.Div(id="result_PA")
        ])

        return new_element_PA
    

@app.callback(
    Output(component_id='result_Claim', component_property='children'),
    [Input(component_id='Drug_Claim', component_property='value'),
     Input(component_id='Payer_Claim', component_property='value')])

def update_WillYouNeedAPA(Drug_Claim_value, Payer_Claim_value):
    
    X = np.array(['B','C','999001','417740','417614'])
    X1 = np.where((X==Drug_Claim_value)|(X==Payer_Claim_value), 1,0)
    proba = Claims_model.predict_proba(X1.reshape(1,-1))
    
    proba_1 = round((proba[0][1]*100),2)

    if (Drug_Claim_value is not None) & (Payer_Claim_value is not None) & (proba_1 < 80): 
        
        reject_code = claims[(claims['bin'].astype(str)==Payer_Claim_value)&(claims['drug']==Drug_Claim_value)&(claims['pharmacy_claim_approved']==0)]['reject_code']
        
        code = np.asarray(reject_code)[0]
        
        return f'''You selected: Drug {Drug_Claim_value} and Payer {Payer_Claim_value}. Probability of your claim approval is {proba_1}%. 
                 You may need a PA. Possible reject code {code}'''
            
    elif (Drug_Claim_value is not None) & (Payer_Claim_value is not None) & (proba_1 > 80) :
        return f'''You selected: Drug {Drug_Claim_value} and Payer {Payer_Claim_value}. Probability of your claim approval is {proba_1}%. 
                     You may not need a PA.'''
    else:
        return ""
    

@app.callback(
    Output(component_id='result_PA', component_property='children'),
    [Input(component_id='Drug_PA', component_property='value'),
     Input(component_id='Payer_PA', component_property='value'),
#      Input(component_id='reject_code', component_property='value'),
     Input(component_id ='Correct_Diagnosis', component_property='value'),
     Input(component_id ='Tried_Failed', component_property='value'),
     Input(component_id ='Contraindication', component_property='value')])

def update_WillPAGetApproved(Drug_PA_value, Payer_PA_value,Correct_Diagnosis_value,Tried_Failed_value,Contraindication_value):
    
    X = np.asarray(['001','010','011','100','101','110','111','AC2','AC3','AC4','BC1','BC2','BC3',
         'BC4','CC1','CC2','CC3','CC4'])
    
    if (Drug_PA_value is not None) & (Payer_PA_value is not None) & (Correct_Diagnosis_value is not None) & (Tried_Failed_value is not None) & (Contraindication_value is not None):
        
        diagnosis = Correct_Diagnosis_value+Tried_Failed_value+Contraindication_value
        drug_payer = Drug_PA_value+Payer_PA_value
        
        X1 = np.where((X== diagnosis) | (X== drug_payer), 1, 0)
        
        proba = PA_model.predict_proba(X1.reshape(1,-1))
    
        proba_1 = round((proba[0][1]*100),2)
    
        return f'''The probability of the PA approval is {proba_1}%'''
    

if __name__ == '__main__':
    Claims_model = joblib.load("../Models/ClaimsModel_logistic.joblib")
    PA_model = joblib.load("../Models/PAModel_logistic.joblib")
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/May/2021 10:56:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 10:56:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 10:56:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 10:56:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 10:56:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 10:56:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 10:56:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 10:56:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 10:56:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 10:56:55] "POST /_dash-update-component HTTP/1.1" 200 -
